In [20]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

ModuleNotFoundError: No module named 'distutils'

In [22]:
# Load the training labels
train_labels_df = pd.read_csv("training_labels.csv")
train_images_dir = "training/"

# Add the directory to the filename
train_labels_df['ID'] = train_labels_df['ID'].apply(lambda x: os.path.join(train_images_dir, x))

# Initialize the ImageDataGenerator for training and validation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

NameError: name 'ImageDataGenerator' is not defined

In [ ]:
# Create the training and validation generators
train_generator = datagen.flow_from_dataframe(
    dataframe=train_labels_df,
    directory=None,  
    x_col='ID',
    y_col='target',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=train_labels_df,
    directory=None,  
    x_col='ID',
    y_col='target',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  
    subset='validation'
)


In [ ]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax')  # Assuming 5 classes of flowers
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # You can adjust the number of epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

In [ ]:
# Load the test data
test_images_dir = "testing/"
test_image_paths = [os.path.join(test_images_dir, f) for f in os.listdir(test_images_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

In [ ]:
# Preprocess test images (resize and rescale)
test_images = np.array([np.array(Image.open(img).resize((224, 224))) / 255. for img in test_image_paths])

In [ ]:
# Generate predictions for the test set
test_predictions = model.predict(test_images)

In [ ]:
# Get class labels from generator
class_labels = list(train_generator.class_indices.keys())

In [ ]:
# Convert predictions to class labels
predicted_labels = [class_labels[np.argmax(pred)] for pred in test_predictions]

In [ ]:
# Generate IDs for the test set
test_ids = [os.path.splitext(os.path.basename(path))[0] for path in test_image_paths]

In [ ]:
# Create a DataFrame for submission
submission_df = pd.DataFrame({'ID': test_ids, 'Prediction': predicted_labels})